In [127]:
import pandas as pd
import numpy as np

In [128]:
modeling_data = pd.read_csv("data_for_modeling_matlab_second_session.csv", index_col=0)

In [129]:
len(modeling_data.mturkID.unique())

114

In [130]:
for n, gp in modeling_data.groupby('mturkID'):
    print(n)
    df_to_save =(gp.reset_index(drop=True)).drop(['mturkID'], axis=1)
    df_to_save['choice'] = df_to_save['choice'].map({'nogo': 2, 'go': 1})
    df_to_save['stimulus'] = df_to_save['stimulus'].map({'go2win': 1, 'go2avoidPun': 2, 'nogo2avoidPun': 3, 'nogo2win': 4})
    df_to_save.to_csv( 'data_for_modeling_second_session/'+ f"{n}.csv", index=False)


A10BH9PYCYUKDJ
A10JXOU89D5RXR
A11S8IAAVDXCUS
A12FTSX85NQ8N9
A12HZGOZQD5YK7
A12X8RUSITV7RU
A136K4YZY2OFTV
A14W0AXTJ3R19V
A173MXK429XAZQ
A1AF25FCVKC87X
A1BM57TUNAQIXM
A1CY7IOJ9YH136
A1DKVUTOBPQH11
A1DZMZTXWOM9MR
A1F1BIPJR11LSR
A1F2APQMJSQVFK
A1FVXS8IM5QYO8
A1FYWSPNO7KN1O
A1IFIK8J49WBER
A1JJYY622DGE5L
A1MJVTR0PCKBWW
A1NLJ1L4VCQYV2
A1PHDT66U6IK4Q
A1R0689JPSQ3OF
A1R8A8BK2VN7RH
A1TW2BZRRS874Z
A1UDOSYE75OYYI
A1W7I6FN183I8F
A1WZY0K6IE3ASG
A20X6V1AAXZGNZ
A2196WCNDZULFS
A21ZYTE6WOKUA
A23KAJRDVCVGOE
A24CG5BTB8KQ39
A24Z9RP5YZZ2TY
A25PFSORDO3SWQ
A2615YW1YERQBO
A26LOVXF4QZZCO
A272X64FOZFYLB
A29AF16RLMI0ES
A2BRZKFG6WAVER
A2DVV59R1CQU6T
A2FGKKWP33DFWS
A2FYFCD16Z3PCC
A2I960JYUZ8KAV
A2IP3ZAFYGV8M9
A2IYR9HF66R029
A2KED6T6BO622W
A2KLJKDG90K1PP
A2ML0070M8FDK1
A2NAKIXS3DVGAA
A2ND52KJ1AIYEK
A2NQR8NLH5P3IH
A2ONILC0LZKG6Y
A2PI6RY9USPIRS
A2PTABUVOUYAH5
A2TLN8489YGY81
A2UGM73EHLK6XI
A2V27A9GZA1NR2
A2VA0NCPWJ8XZG
A2WPKP73S4MBLK
A2XS49LRNX8PD4
A2YTO4EY3MNYAJ
A34YDGVZKRJ0LZ
A353XKDTUX10NC
A358YR7MWC6LRH
A35TUIBF05D

In [90]:
modeling_data

,mturkID,choice,feedback_points,stimulus
85,A3E8SXH0BAYG85,go,1,go2win
91,A3E8SXH0BAYG85,nogo,-1,go2avoidPun
97,A3E8SXH0BAYG85,nogo,1,nogo2win
103,A3E8SXH0BAYG85,nogo,-1,nogo2avoidPun
109,A3E8SXH0BAYG85,go,-1,nogo2avoidPun
...,...,...,...,...
108270,A20X6V1AAXZGNZ,go,0,go2avoidPun
108276,A20X6V1AAXZGNZ,nogo,1,nogo2win
108282,A20X6V1AAXZGNZ,go,-1,go2avoidPun
108288,A20X6V1AAXZGNZ,go,1,go2win


In [131]:
import scipy.io 
model_data_second = scipy.io.loadmat('modelling_data_second_session/second210228_EM_-llbaepxb-ite8.mat')
participant_data_second = scipy.io.loadmat('modelling_data_second_session/gonogo_data_second_session.mat')
model_result_df_second = {}
model_result_df_second['mturkID'] = []
for i in range(114):

    model_result_df_second['mturkID'].append((participant_data_second['subject_ids'][0][i][0]))
model_result_df_second['alpha'] = 1/(1+np.exp(-model_data_second['E'][1]))
model_result_df_second['beta'] = np.exp(model_data_second['E'][0])
model_result_df_second['noise'] = 1/(1+np.exp(-model_data_second['E'][3]))
model_result_df_second['bias'] = model_data_second['E'][4]
model_result_df_second['pav'] = np.exp(model_data_second['E'][2])
pd.DataFrame(model_result_df_second).to_csv("result_of_modeling_matlab_second_session.csv", index=False)


In [132]:
model_data_first = scipy.io.loadmat('modelling_data_first_session/first210227_EM_-llbaepxb-ite1.mat')
participant_data_first = scipy.io.loadmat('modelling_data_first_session/gonogo_data_first_session.mat')
model_result_df_first = {}
model_result_df_first['mturkID'] = []
for i in range(146):

    model_result_df_first['mturkID'].append((participant_data_first['subject_ids'][0][i][0]))
model_result_df_first['alpha'] = 1/(1+np.exp(-model_data_first['E'][1]))
model_result_df_first['beta'] = np.exp(model_data_first['E'][0])
model_result_df_first['noise'] = 1/(1+np.exp(-model_data_first['E'][3]))
model_result_df_first['bias'] = model_data_first['E'][4]
model_result_df_first['pav'] = np.exp(model_data_first['E'][2])
pd.DataFrame(model_result_df_first)


,mturkID,alpha,beta,noise,bias,pav
0,A10BH9PYCYUKDJ,0.032798,8.745182,0.977425,0.111071,2.074325
1,A10JXOU89D5RXR,0.739849,9.161395,0.949481,3.194171,0.621218
2,A10M63DSIGIEDI,0.090104,1.148812,0.983119,3.916653,0.201712
3,A11S8IAAVDXCUS,0.333036,48.601867,0.985864,1.305883,0.350864
4,A12FTSX85NQ8N9,0.443714,2.131303,0.970367,-0.874009,0.282252
...,...,...,...,...,...,...
141,AW5O1RK3W60FC,0.343936,10.715497,0.981412,1.226195,0.105567
142,AXAO7UJYYEFCO,0.124538,5.070514,0.692038,1.648907,0.072937
143,AYW62R027PUT1,0.137548,9.017771,0.978309,0.328262,0.139512
144,AZ69TBTDH7AZS,0.170632,26.772653,0.904668,1.021437,1.459342
